In [1]:
import numpy as np
import time
import os
from tqdm import tqdm
import pyarrow.parquet as pq
from sklearn.neighbors import KDTree
# --- ИСПРАВЛЕНИЕ: Сначала включаем экспериментальную функцию ---
from sklearn.experimental import enable_iterative_imputer
# --- Затем импортируем ее ---
from sklearn.impute import IterativeImputer
import json

# --- Configuration ---
DATA_FILE = "ad_features_all_gt1_14_08.parquet"
OUTPUT_FILE = "tde_nearest_neighbors_results_all_with_em_imputation.json"

# Target Object IDs for different categories can be uncommented as needed.
# TDE
TARGET_OBJECT_IDS = ['ZTF22abegjtx', 'ZTF22abkfhua', 'ZTF23aapyidj', 'ZTF23abohtqf', 'ZTF24aaecooj', 'ZTF24aajvvhj', 'ZTF24aakaiha', 'ZTF24aamfius', 'ZTF24aatxshz', 'ZTF24abfaake']
# SLNS
#TARGET_OBJECT_IDS = ['ZTF25aanxtou', 'ZTF25aaixrfr', 'ZTF24abvftmi', 'ZTF24abdhylt', 'ZTF24aaysowl', 'ZTF23aboebgh', 'ZTF23abcvbqq', 'ZTF23aanptpp', 'ZTF22abcvfgs', 'ZTF21ackxdos']
# CV
#TARGET_OBJECT_IDS = ['ZTF18abjcuhv', 'ZTF18aaudzmj', 'ZTF19aagxcga', 'ZTF18aaashju', 'ZTF21aagtpxy', 'ZTF18aboacjl', 'ZTF17aaantxj', 'ZTF18aajwgru', 'ZTF18abigrzf', 'ZTF18adibadz']

NUM_NEIGHBORS = 10

FEATURE_NAMES = [
    'mean_g', 'weighted_mean_g',
    'standard_deviation_g', 'median_g', 'amplitude_g', 'beyond_1_std_g',
    'cusum_g', 'inter_percentile_range_10_g', 'kurtosis_g', 'linear_trend_g',
    'linear_trend_sigma_g', 'linear_trend_noise_g', 'linear_fit_slope_g',
    'linear_fit_slope_sigma_g', 'linear_fit_reduced_chi2_g',
    'magnitude_percentage_ratio_40_5_g', 'magnitude_percentage_ratio_20_10_g',
    'maximum_slope_g', 'median_absolute_deviation_g',
    'median_buffer_range_percentage_10_g', 'percent_amplitude_g',
    'anderson_darling_normal_g', 'chi2_g', 'skew_g', 'stetson_K_g', 'mean_r',
    'weighted_mean_r', 'standard_deviation_r', 'median_r', 'amplitude_r',
    'beyond_1_std_r', 'cusum_r', 'inter_percentile_range_10_r', 'kurtosis_r',
    'linear_trend_r', 'linear_trend_sigma_r', 'linear_trend_noise_r',
    'linear_fit_slope_r', 'linear_fit_slope_sigma_r', 'linear_fit_reduced_chi2_r',
    'magnitude_percentage_ratio_40_5_r', 'magnitude_percentage_ratio_20_10_r',
    'maximum_slope_r', 'median_absolute_deviation_r',
    'median_buffer_range_percentage_10_r', 'percent_amplitude_r',
    'anderson_darling_normal_r', 'chi2_r', 'skew_r', 'stetson_K_r', 'distnr'
]

def process_data_from_parquet(filepath):
    """
    Loads data from a Parquet file, imputes missing values using an EM-like
    algorithm (IterativeImputer), and then normalizes the data.
    """
    print(f"Step 1: Reading data from '{filepath}'...")
    try:
        pq_file = pq.ParquetFile(filepath)
    except Exception as e:
        print(f" ERROR: Could not open Parquet file '{filepath}'. {e}"); return None, None

    num_objects_initial = pq_file.metadata.num_rows
    print(f"   The file contains {num_objects_initial:,} objects.")

    object_ids_list, feature_batches = [], []
    columns_to_read = ['objectId'] + FEATURE_NAMES

    for i in tqdm(range(pq_file.num_row_groups), desc="Reading Parquet batches"):
        batch = pq_file.read_row_group(i, columns=columns_to_read)
        object_ids_list.extend(batch.column('objectId').to_pylist())
        feature_batches.append(np.column_stack([batch.column(name).to_numpy() for name in FEATURE_NAMES]))

    print("Combining batches...")
    data_matrix = np.vstack(feature_batches).astype('float32')
    object_ids = np.array(object_ids_list)
    print(f" Data loaded. Matrix size: {data_matrix.shape}")

    print("\nStep 2: Imputing missing values (NaN) using IterativeImputer (EM-like approach)...")
    total_nan_values = np.isnan(data_matrix).sum()

    if total_nan_values > 0:
        print(f"   Found {total_nan_values:,} missing values (NaN) to impute.")
        
        # IterativeImputer is a multivariate imputer that models each feature with
        # missing values as a function of other features. It's a robust method
        # similar in spirit to the EM algorithm for this kind of task.
        imputer = IterativeImputer(max_iter=10, random_state=42, verbose=2)
        
        print("   Starting imputation process (this may take a while)...")
        start_time = time.time()
        data_matrix = imputer.fit_transform(data_matrix)
        end_time = time.time()
        
        print(f"   Imputation complete in {end_time - start_time:.2f} seconds.")
        print(f"   Matrix size remains: {data_matrix.shape}")
    else:
        print("   No missing values (NaN) found. Skipping imputation.")

    print("\nStep 3: Normalizing data (Standardization)...")
    mean = np.mean(data_matrix, axis=0, dtype='float32')
    std = np.std(data_matrix, axis=0, dtype='float32')
    std[std == 0] = 1.0  # Avoid division by zero for constant features
    data_matrix = (data_matrix - mean) / std
    print("   Data successfully normalized.")

    return object_ids.tolist(), data_matrix

def find_neighbors_kdtree_batch(data, query_vectors, k_candidates):
    """
    Builds a KD-Tree on the CPU and performs a batch search.
    """
    print("\nStep 4: Building KD-Tree...")
    print("   (This may take a few minutes depending on data size and CPU power)")
    start_time = time.time()
    kdt = KDTree(data, leaf_size=40, metric='euclidean')
    end_time = time.time()
    print(f"   Tree built in {end_time - start_time:.2f} sec.")

    print(f"\n   Performing batch search for {len(query_vectors)} objects...")
    start_time = time.time()
    distances, indices = kdt.query(query_vectors, k=k_candidates)
    end_time = time.time()
    print(f"   Search completed in {end_time - start_time:.4f} sec.")

    return distances, indices

# --- Main execution block ---
if __name__ == "__main__":
    if not os.path.exists(DATA_FILE):
        print(f"ERROR: Data file '{DATA_FILE}' not found.")
        exit()

    all_ids, norm_data = process_data_from_parquet(DATA_FILE)
    if all_ids is None:
        exit()

    print("\nStep 5: Finding target objects and preparing query vectors...")
    id_to_idx = {oid: i for i, oid in enumerate(all_ids)}

    query_vectors_list, found_target_ids = [], []
    for target_id in TARGET_OBJECT_IDS:
        if target_id in id_to_idx:
            query_vectors_list.append(norm_data[id_to_idx[target_id]])
            found_target_ids.append(target_id)
        else:
            # This is less likely to happen now, but good to keep as a safeguard
            print(f"   Warning: Target object '{target_id}' not found in the original Parquet file and will be skipped.")

    if not found_target_ids:
        print("ERROR: None of the target objects were found in the data. Exiting.")
        exit()

    query_vectors = np.array(query_vectors_list, dtype='float32')
    print(f"   Found {len(found_target_ids)} target objects. Preparing to search.")

    distances_raw, indices_raw = find_neighbors_kdtree_batch(norm_data, query_vectors, NUM_NEIGHBORS + 1)

    print("\n" + "="*80)
    print("NEAREST NEIGHBOR SEARCH RESULTS (KD-Tree, data imputed with IterativeImputer)")
    print("="*80)

    results_to_save = {}

    for i, target_id in enumerate(found_target_ids):
        final_neighbors_for_print = []
        final_neighbor_ids_for_json = []
        # The target object itself will be the first result (distance 0), so we find its unique neighbors.
        seen_ids = {target_id}

        distances_for_target, indices_for_target = distances_raw[i], indices_raw[i]

        for j in range(len(indices_for_target)):
            if len(final_neighbor_ids_for_json) >= NUM_NEIGHBORS:
                break

            neighbor_id = all_ids[indices_for_target[j]]
            if neighbor_id in seen_ids:
                continue
            
            final_neighbor_ids_for_json.append(neighbor_id)
            
            neighbor_dist = distances_for_target[j]
            final_neighbors_for_print.append({'id': neighbor_id, 'distance': neighbor_dist})
            
            seen_ids.add(neighbor_id)
        
        results_to_save[target_id] = final_neighbor_ids_for_json

        print(f"\n--- {NUM_NEIGHBORS} nearest unique neighbors for: {target_id} ---\n")
        print(f"{'Rank':<5} {'ObjectID':<20} {'L2 Distance':<20}")
        print("-"*50)
        for rank, neighbor in enumerate(final_neighbors_for_print, 1):
            print(f"{rank:<5} {neighbor['id']:<20} {neighbor['distance']:<20.4f}")

    print("\n" + "="*80)
    print(f"Saving results to file '{OUTPUT_FILE}'...")
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(results_to_save, f, indent=4)
        print(f"Results successfully saved.")
    except Exception as e:
        print(f"ERROR: Could not save the file. {e}")

    print("\n" + "="*80)

Step 1: Reading data from 'ad_features_all_gt1_14_08.parquet'...
   The file contains 3,922,520 objects.


Reading Parquet batches: 100%|████████████████████| 1/1 [00:07<00:00,  7.77s/it]


Combining batches...
 Data loaded. Matrix size: (3922520, 51)

Step 2: Imputing missing values (NaN) using IterativeImputer (EM-like approach)...
   Found 102,253,461 missing values (NaN) to impute.
   Starting imputation process (this may take a while)...
[IterativeImputer] Completing matrix with shape (3922520, 51)
[IterativeImputer] Ending imputation round 1/10, elapsed time 320.40
[IterativeImputer] Change: 69401.0625, scaled tolerance: 52.08502960205078 
[IterativeImputer] Ending imputation round 2/10, elapsed time 624.70
[IterativeImputer] Change: 214316.40625, scaled tolerance: 52.08502960205078 
[IterativeImputer] Ending imputation round 3/10, elapsed time 938.77
[IterativeImputer] Change: 291801.96875, scaled tolerance: 52.08502960205078 
[IterativeImputer] Ending imputation round 4/10, elapsed time 1251.22
[IterativeImputer] Change: 1085525.625, scaled tolerance: 52.08502960205078 
[IterativeImputer] Ending imputation round 5/10, elapsed time 1566.48
[IterativeImputer] Change

/home/home/projects/fink/.venv/lib/python3.13/site-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


   Imputation complete in 3513.21 seconds.
   Matrix size remains: (3922520, 51)

Step 3: Normalizing data (Standardization)...
   Data successfully normalized.

Step 5: Finding target objects and preparing query vectors...
   Found 10 target objects. Preparing to search.

Step 4: Building KD-Tree...
   (This may take a few minutes depending on data size and CPU power)
   Tree built in 31.69 sec.

   Performing batch search for 10 objects...
   Search completed in 0.2278 sec.

NEAREST NEIGHBOR SEARCH RESULTS (KD-Tree, data imputed with IterativeImputer)

--- 10 nearest unique neighbors for: ZTF22abegjtx ---

Rank  ObjectID             L2 Distance         
--------------------------------------------------
1     ZTF18abeocrt         0.2748              
2     ZTF18abncwah         0.2926              
3     ZTF22aafpgzk         0.2953              
4     ZTF18abdlrnf         0.3467              
5     ZTF22aanaxtu         0.3958              
6     ZTF18abesbws         0.4433            